### Import the lib needed

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

### Get the data

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- each row corresponding one training example
- have label and pixel value [0 - 783]

### Load the data

In [4]:
data = np.array(data)

# get the dimention of the data
m, n = data.shape
np.random.shuffle(data)

# get the tram of the data -> each column is one data
data_dev = data[0: 1000].T

Y_dev = data_dev[0]
X_dev = data_dev[1: n]

# training data
data_train = data[1000: m].T
Y_train = data_train[0]
X_train = data_train[1: n]

In [5]:
# check
X_train[:, 0].shape

(784,)

### Initialize parameters

In [24]:
def init_params(size):
    W1 = np.random.rand(10,784) - 0.5
    b1 = np.random.rand(10,size) - 0.5
    W2 = np.random.rand(10,10) - 0.5
    b2 = np.random.rand(10,size) - 0.5
    return W1, b1, W2, b2

### Define the function for Forward Propagation

In [19]:
def ReLU(Z):
    return np.maximum(Z, 0)


def softmax(Z):
    # print('The value of the Z is',Z)
    A = np.exp(Z) / sum(np.exp(Z))
    # print('The value of the A is',A)
    return A

In [8]:
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

### Define the function for Backward Propagation

In [9]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [10]:
def deriv_ReLU(Z):
    return Z > 0

In [29]:
def backward_prop(Z1, A1, Z2, A2, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = (1 / m) * dZ2.dot(A1.T)
    db2 = (1 / m) * np.sum(dZ2, 1)

    dZ1 = W2.T.dot(dZ2 * deriv_ReLU(Z1))
    dW1 = (1 / m) * dZ1.dot(X.T)
    db1 = (1 / m) * np.sum(dZ1, 1)
    return dW1, db1, dW2, db2

In [27]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * np.reshape(db1, (10,1))
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * np.reshape(db2, (10,1))
    return W1, b1, W2, b2

In [13]:
def get_predictions(A2):
    return np.argmax(A2, 0)


def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

In [25]:
def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params(Y.size)
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 20 == 0:
            print('iterations:', i)
            print('Accuracy:', get_accuracy(get_predictions(A2), Y))
    return W1, b1, W2, b2

In [30]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 200, 0.1)

/var/folders/df/mx2gggdn7p55_8g8_fxxk8zc0000gn/T/ipykernel_9137/1916436928.py:7: RuntimeWarning: overflow encountered in exp
  A = np.exp(Z) / sum(np.exp(Z))
/var/folders/df/mx2gggdn7p55_8g8_fxxk8zc0000gn/T/ipykernel_9137/1916436928.py:7: RuntimeWarning: overflow encountered in add
  A = np.exp(Z) / sum(np.exp(Z))
/var/folders/df/mx2gggdn7p55_8g8_fxxk8zc0000gn/T/ipykernel_9137/1916436928.py:7: RuntimeWarning: invalid value encountered in true_divide
  A = np.exp(Z) / sum(np.exp(Z))


iterations: 0
[1 0 1 ... 1 3 3] [0 6 1 ... 5 3 3]
Accuracy: 0.1660487804878049
iterations: 20
[0 0 0 ... 0 0 0] [0 6 1 ... 5 3 3]
Accuracy: 0.09829268292682927


KeyboardInterrupt: 

In [20]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions


def test_predictions(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print('Prediction:', prediction)
    print('Label:', label)

    current_image = current_image.reshape((28,28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation = 'nearest')
    plt.show()

In [ ]:
test_predictions(5, W1, b1, W2, b2)

In [ ]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)